In [42]:
import logging
from collections import namedtuple
import random
from typing import Callable
from copy import deepcopy
from itertools import accumulate
from operator import xor

In [43]:
Nimply = namedtuple("Nimply", "row, num_objects")

class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    @property
    def k(self) -> int:
        return self._k

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects

In [44]:
def nim_sum(state: Nim) -> int:
    *_, result = accumulate(state.rows, xor)
    return result

def cook_status(state: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = [
        (r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1) if state.k is None or o <= state.k
    ]
    cooked["active_rows_number"] = sum(o > 0 for o in state.rows)
    cooked["shortest_row"] = min((x for x in enumerate(state.rows) if x[1] > 0), key=lambda y: y[1])[0]
    cooked["longest_row"] = max((x for x in enumerate(state.rows)), key=lambda y: y[1])[0]
    cooked["nim_sum"] = nim_sum(state)

    brute_force = list()
    for m in cooked["possible_moves"]:
        tmp = deepcopy(state)
        tmp.nimming(m)
        brute_force.append((m, nim_sum(tmp)))
    cooked["brute_force"] = brute_force

    return cooked

In [45]:
def optimal_startegy(state: Nim) -> Nimply:
    data = cook_status(state)
    return next((bf for bf in data["brute_force"] if bf[1] == 0), random.choice(data["brute_force"]))[0]

def pure_random(state: Nim) -> Nimply:
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)

In [46]:
logging.getLogger().setLevel(logging.DEBUG)

strategy = (pure_random, optimal_startegy)

nim = Nim(11)
logging.debug(f"status: Initial board  -> {nim}")
player = 0
while nim:
    ply = strategy[player](nim)
    nim.nimming(ply)
    logging.debug(f"status: After player {player} -> {nim}")
    player = 1 - player
winner = 1 - player
logging.info(f"status: Player {winner} won!")

DEBUG:root:status: Initial board  -> <1 3 5 7 9 11 13 15 17 19 21>
DEBUG:root:status: After player 0 -> <1 3 5 7 9 4 13 15 17 19 21>
DEBUG:root:status: After player 1 -> <1 3 5 7 9 4 13 15 9 19 21>
DEBUG:root:status: After player 0 -> <1 2 5 7 9 4 13 15 9 19 21>
DEBUG:root:status: After player 1 -> <0 2 5 7 9 4 13 15 9 19 21>
DEBUG:root:status: After player 0 -> <0 2 5 7 9 1 13 15 9 19 21>
DEBUG:root:status: After player 1 -> <0 2 0 7 9 1 13 15 9 19 21>
DEBUG:root:status: After player 0 -> <0 1 0 7 9 1 13 15 9 19 21>
DEBUG:root:status: After player 1 -> <0 1 0 4 9 1 13 15 9 19 21>
DEBUG:root:status: After player 0 -> <0 0 0 4 9 1 13 15 9 19 21>
DEBUG:root:status: After player 1 -> <0 0 0 4 8 1 13 15 9 19 21>
DEBUG:root:status: After player 0 -> <0 0 0 4 8 0 13 15 9 19 21>
DEBUG:root:status: After player 1 -> <0 0 0 4 8 0 12 15 9 19 21>
DEBUG:root:status: After player 0 -> <0 0 0 2 8 0 12 15 9 19 21>
DEBUG:root:status: After player 1 -> <0 0 0 2 8 0 10 15 9 19 21>
DEBUG:root:status: Aft